# Prepare geom90m data (Amatulli)

In [ ]:
# libraries
import os, time, shutil, rioxarray
import numpy as np
import xarray as xr
from scipy import ndimage
from dask.distributed import Lock
import matplotlib.pyplot as plt

In [ ]:
# Directories
dir_data =  '../data/'
dir01 = '../paper_deficit/output/01_prep/'

---

### Pre-processing

In [ ]:
# Libraries
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
import dask

# Initialize dask
cluster = SLURMCluster(
    queue='compute',                      # SLURM queue to use
    cores=24,                             # Number of CPU cores per job
    memory='256 GB',                      # Memory per job
    account='bm0891',                     # Account allocation
    interface="ib0",                      # Network interface for communication
    walltime='00:30:00',                  # Maximum runtime per job
    local_directory='../dask/',           # Directory for local storage
    job_extra_directives=[                # Additional SLURM directives for logging
        '-o ../dask/LOG_worker_%j.o',     # Output log
        '-e ../dask/LOG_worker_%j.e'      # Error log
    ]
)

# Scale dask cluster
cluster.scale(jobs=10)

# Configurate dashboard url
dask.config.config.get('distributed').get('dashboard').update(
    {'link': '{JUPYTERHUB_SERVICE_PREFIX}/proxy/{port}/status'}
)

# Create client
client = Client(cluster)

client

In [ ]:
# Function to trim memory of workers
def trim_memory() -> int:
    import ctypes
    libc = ctypes.CDLL("libc.so.6")
    return libc.malloc_trim(0)

In [ ]:
def prep_geom90m():
    
    """Prepare geom90m files for regridding"""

    # list of files
    list_files = [i for i in os.listdir(dir_data + 'geomorpho90m/') 
                  if 'tif' in i]
    
    for i in list_files:
        # file
        f_str = (dir_data + 'geomorpho90m/' + i)
        # file name
        f_name = i.split('_')[1]
        
        # read file, rename, export as tif
        da = rioxarray.open_rasterio(f_str, chunks = dict(y=10000, x=10000)) \
            .rename('geom90m_' + f_name) \
            .drop_vars('spatial_ref') \
            .rio.to_raster(dir01 + 'geom90m_' + f_name + '.tif')
        
        # release memory
        client.run(trim_memory);

%time prep_geom90m()

In [ ]:
# Close dask cluster
cluster.close()

In [ ]:
# Wait for 60s for dask client to completely disconnect
time.sleep(60)

---

### Regridding

In [ ]:
# Import regridding function
from regrid_high_res_v1_01 import regrid_high_res, prep_tif

In [ ]:
def regrid_da(f_source, dir_target, dir_source, dir_out, 
              size_tiles, fill_value=None, olap=1):  
    """Regrid large xarray dataarrays.

    Args:
        f_source (str): The filename (without extension) of the source .tif file to be regridded.
        dir_target (str): Directory containing target grid .tif file.
        dir_source (str): Directory containing the the source  .tif file.
        dir_out (str): Directory to store the output and intermediate files.
        size_tiles (int): Size of the regridding tiles in degrees.
        fill_value (float, optional): Fill value to use in the regridding process. Defaults to None.
        olap (int, optional): Overlap size in degrees for regridding tiles. Defaults to 1.
        
    Returns:
        xarray.Dataset: The combined dataset after regridding.
    """
    # Prepare the target and source data arrays from TIFF files
    da_target = prep_tif(dir_target + 'target_grid.tif', 'target_grid')
    da_source = prep_tif(dir_source + f_source + '.tif', f_source)
    # Regridd source array to target grid
    regrid_high_res(da_target, da_source, dir_out,
                    account='bm0891', partition='compute',
                    size_tiles=size_tiles, olap=olap, fill_value = fill_value,
                    type_export='zarr', del_interm=False)

In [ ]:
# list with prepared geom90m variable names
list_geom90m = ['geom90m_northness', 'geom90m_slope',
                  'geom90m_cti', 'geom90m_convergence',
                  'geom90m_eastness', 'geom90m_spi']
list_geom90m = ['geom90m_eastness', 'geom90m_spi']
for i in list_geom90m:
    print(i)
    if i == 'geom90m_spi':
        %time regrid_da(i, dir01, dir01, dir01, 20, 65535)
    else:
        %time regrid_da(i, dir01, dir01, dir01, 20, -32768)

---

### Fill nans

In [ ]:
def fill_nans(var, dir_out):
    """
    Fills NaN values in the specified variable's dataset using the nearest valid 
    data, applies a land mask, and exports the result to a new Zarr dataset.

    Args:
        var (str): Name of the variable to process (e.g., 'temperature', 'precipitation').
        dir_out (str): Directory where prepared data is stored and the filled dataset will be exported.

    Returns:
        None
    """

    def fill_nans_array(data, invalid):
        """
        Replace invalid (NaN) data cells by the value of the nearest valid data 
        cell.
        """
        ind = ndimage.distance_transform_edt(invalid,
                                             return_distances=False,
                                             return_indices=True)
        return data[tuple(ind)]

    # Paths for input and output
    land_mask_path = os.path.join(dir_out, 'ds_prep_copernicus_land_mask.zarr')
    var_data_path = os.path.join(dir_out, f'ds_regridded_{var}.zarr')
    output_path = os.path.join(dir_out, f'ds_prep_{var}.zarr')

    # Read land mask data
    da_land = xr.open_zarr(land_mask_path) \
                .chunk(dict(lat=5000, lon=5000)) \
                .copernicus_land_mask \
                .compute()

    # Read variable data
    da_var = xr.open_zarr(var_data_path)['regridded_' + var]

    # Fill nan using function fill_nans_array
    # If there are no NaNs, skip filling process
    if not da_var.isnull().any():
        da_fill = da_var.values  # No filling required
    else:
        da_fill = fill_nans_array(da_var.values, da_var.isnull().values)

    # Create a new Dataset with filled data
    ds_filled = xr.Dataset(dict(lat = da_var.lat, lon=da_var.lon))
    ds_filled[var] = (('lat', 'lon'), da_fill)

    # Apply land mask to the filled data
    ds_filled = ds_filled.where(da_land)

    # Export the filled dataset to Zarr format
    ds_filled.chunk(dict(lat=5000, lon=5000)) \
             .to_zarr(output_path, mode='w')

In [ ]:
%%time
# list with prepared geom90m variable names
list_geom90m = ['geom90m_northness', 'geom90m_slope',
                  'geom90m_cti', 'geom90m_convergence',
                  'geom90m_eastness', 'geom90m_spi']

for i in list_geom90m:
    %time fill_nans(i, dir01)

---

### Checks

In [ ]:
# Plot to check
# List with prepared geom90m variable names
list_geom90m = ['geom90m_northness', 'geom90m_slope',
                  'geom90m_cti', 'geom90m_convergence',
                  'geom90m_eastness', 'geom90m_spi']

# Plot
for i in list_geom90m:
    fig, ax = plt.subplots(figsize=(10, 5), ncols=1, nrows=1)
    xr.open_zarr(dir01 + 'ds_prep_' + i + '.zarr')[i] \
        .plot.imshow(ax=ax)
    ax.set_title(i)